In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader


In [3]:
mean = [0.5071, 0.4867, 0.4408]
std  = [0.2675, 0.2565, 0.2761]

transform = transforms.Compose([
  
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


In [4]:
train_data = datasets.CIFAR100(root='./data', train=True, download=False, transform=transform)
test_data = datasets.CIFAR100(root='./data', train=False, download=False, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, downsample=False):
        super(ResidualBlock, self).__init__()
        stride = 2 if downsample else 1

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = None
        if downsample or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.downsample:
            identity = self.downsample(identity)

        out = out + identity
        out = self.relu(out)
        return out


In [6]:
class ClassifierHead(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(ClassifierHead, self).__init__()
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_dim, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.classifier(x)

In [7]:
class DashNet18(nn.Module):
    def __init__(self, num_classes=100):
        super(DashNet18, self).__init__()
        self.layer0 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )

        self.layer1 = nn.Sequential(
            ResidualBlock(64, 64),
            ResidualBlock(64, 64)
        )
        self.layer2 = nn.Sequential(
            ResidualBlock(64, 128, downsample=True),
            ResidualBlock(128, 128)
        )
        self.layer3 = nn.Sequential(
            ResidualBlock(128, 256, downsample=True),
            ResidualBlock(256, 256)
        )
        self.layer4 = nn.Sequential(
            ResidualBlock(256, 512, downsample=True),
            ResidualBlock(512, 512)
        )

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = ClassifierHead(input_dim=512, num_classes=num_classes)

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DashNet18(num_classes=100).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)


In [ ]:
epochs = 100

for epoch in range(epochs):
    # Training
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100. * correct / total
    avg_train_loss = train_loss / len(train_loader)

    # Evaluation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_accuracy = 100. * val_correct / val_total
    avg_val_loss = val_loss / len(test_loader)

    # Print epoch summary
    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%")
    print(f"Val   Loss: {avg_val_loss:.4f}, Val   Acc: {val_accuracy:.2f}%\n")


Epoch [1/100]
Train Loss: 4.1702, Train Acc: 4.87%
Val   Loss: 4.1125, Val   Acc: 5.32%

Epoch [2/100]
Train Loss: 3.8958, Train Acc: 7.97%
Val   Loss: 3.7932, Val   Acc: 9.62%

Epoch [3/100]
Train Loss: 3.6568, Train Acc: 11.74%
Val   Loss: 3.6011, Val   Acc: 13.15%

Epoch [4/100]
Train Loss: 3.4214, Train Acc: 15.44%
Val   Loss: 3.3045, Val   Acc: 17.57%

Epoch [5/100]
Train Loss: 3.2192, Train Acc: 19.23%
Val   Loss: 3.0282, Val   Acc: 21.41%

Epoch [6/100]
Train Loss: 3.0358, Train Acc: 22.28%
Val   Loss: 2.8012, Val   Acc: 26.71%

Epoch [7/100]
Train Loss: 2.8756, Train Acc: 25.77%
Val   Loss: 2.7975, Val   Acc: 27.58%

Epoch [8/100]
Train Loss: 2.7576, Train Acc: 28.00%
Val   Loss: 2.6055, Val   Acc: 30.07%

Epoch [9/100]
Train Loss: 2.6396, Train Acc: 30.32%
Val   Loss: 2.4386, Val   Acc: 35.07%

Epoch [10/100]
Train Loss: 2.5364, Train Acc: 32.62%
Val   Loss: 2.4286, Val   Acc: 35.37%

Epoch [11/100]
Train Loss: 2.4500, Train Acc: 34.49%
Val   Loss: 2.4572, Val   Acc: 34.80%

E